# Code

In [ ]:
import weave

In [ ]:
def simple_map_prepare(nCh, nPix):
    """
    Prepare a simple map.
    
    Parameters
    ----------
    nCh: int
        Number of channels
    nPix: int
        Number of pixels
    
    Returns
    -------
    signal_map: ndarray
        an array of length nCh, where each entry is a length-nPix zero array.
    hits_map: ndarray
        same as above with dtype int32
    """
    signal_map = np.zeros((nCh, nPix))
    hits_map = np.zeros((nCh, nPix), dtype=np.int32)
    return signal_map, hits_map

In [ ]:
def simple_map(signal_map,hits_map,pointing,mask,array):
	''' Map timestream into a signal map '''
	nch,nt = array.shape
	npix = signal_map.shape[1]
	assert signal_map.dtype == array.dtype
	assert mask.dtype == np.bool
	
	c_code = '''
	int ch,t;
	int pix;
	for(ch=0;ch<nch;ch++) {
		for(t=0;t<nt;t++) {
			pix = pointing(t);
			signal_map(ch,pix) += array(ch,t)*mask(ch,t);
			hits_map(ch,pix) += mask(ch,t);
		}
	}

	for(ch=0;ch<nch;ch++) {
		for(pix=0;pix<npix;pix++) {
			if(hits_map(ch,pix) > 0) {
				signal_map(ch,pix) /= hits_map(ch,pix);
			}
		}
	}
	'''

	weave.inline(c_code,['array','pointing','mask','signal_map','hits_map','nch','nt','npix'],type_converters=weave.converters.blitz)

In [ ]:
def simple_scan_subtract(signal_map,pointing,array):
	''' Directly subtract signal map from array '''
	nch,nt = array.shape
	npix = signal_map.shape[1]

	assert np.max(pointing) < npix
	assert np.min(pointing) >= 0

	c_code = '''
	int ch,t;
	int pix;
	for(ch=0;ch<nch;ch++) {
		for(t=0;t<nt;t++) {
			pix = pointing(t);
			array(ch,t) -= signal_map(ch,pix);
		}
	}
	'''

	weave.inline(c_code,['array','pointing','npix','nt','nch','signal_map'],type_converters=weave.converters.blitz)

In [ ]:
def ground_template_filter_array(
        input_array,
        az,
        mask,
        pixel_size,
        groundmap=False,
        lr=False,
        filtmask=None):
    '''
    Remove ground template from array timestreams

    Parameters
    ----------
    input_array: array_like
        shape: (number of channels, number of time steps)
        Input timestream, mutated inplace.
    az: array_like
        shape: input_array[0]
        The azimuth of the timestream.
    mask: array_like
        shape: input_array
        dtype: bool
    pixel_size: float
    groundmap: bool
        If groundmap = True, then do the exact opposite,
        and remove component from timestream that isn't fixed with the ground
    lr: bool
        If true, ground substraction done separately on left and right moving scans
    filtmask: array_like
        shape: input_array
        dtype: bool
        filtmask means to compute the filter template with that subset of the data,
        operation is applied to data specified by mask
        In largepatch filtmask refers to wafermask_chan_filt
    '''
    # initialize
    nCh, nTime = input_array.shape
    az_min = np.min(az)
    az_max = np.max(az)
    az_range = az_max - az_min
    if filtmask is None:
        filtmask = mask

    assert input_array.shape == mask.shape == filtmask.shape
    assert az.size == nTime

    # Calculate number of pixels given the pixel size
    nPix = int(np.round(az_range / pixel_size))
    assert nPix > 3
    # recalculate pixel_size (because nPix is int)
    pixel_size = az_range / nPix

    # bin at nPix is used as a junk pixel
    signal_map, hits_map = simple_map_prepare(nCh, nPix + 1)

    # get pointing
    pointing = np.int_(np.floor(nPix * (az - az_min) / az_range))
    # One point with az = az_max will end up one bin too far left
    pointing[pointing == nPix] = nPix - 1

    if groundmap:
        array_in = input_array.copy()

    if lr:
        vaz = np.gradient(az)
        # select left / right moving timestream
        l = vaz >= 0
        r = vaz < 0
        pointingL = pointing.copy()
        pointingL[~l] = nPix
        pointingR = pointing.copy()
        pointingR[~r] = nPix

        simple_map(signal_map, hits_map, pointingL, filtmask, input_array)
        signal_map[:, nPix] = 0
        simple_scan_subtract(signal_map, pointingL, input_array)
        signal_map[:] = 0
        hits_map[:] = 0
        simple_map(signal_map, hits_map, pointingR, filtmask, input_array)
        signal_map[:, nPix] = 0
        simple_scan_subtract(signal_map, pointingR, input_array)
    else:
        simple_map(signal_map, hits_map, pointing, filtmask, input_array)
        simple_scan_subtract(signal_map, pointing, input_array)

    if groundmap:
        input_array[:, :] = array_in - input_array

# Tests

In [ ]:
import numpy as np
from numpy.testing import assert_allclose

def assertIdenticalList(list1, list2, rtol):
    for i, list1i in enumerate(list1):
        if isinstance(list1i, bool):
            assert list1i is list2[i]
        else:
            assert_allclose(list1i, list2[i], rtol)

In [ ]:
import pickle
import sys
py2 = sys.version_info[0] == 2


def test_ground_template_filter_array(rtol=1e-7):
    with open('tests/timestream/ground_template_filter_array_input.pkl', 'rb') as f:
        if not py2:
            ground_template_filter_array_input = pickle.load(
                f, encoding='latin1')
        else:
            ground_template_filter_array_input = pickle.load(f)
    with open('tests/timestream/ground_template_filter_array_output.pkl', 'rb') as f:
        if not py2:
            ground_template_filter_array_output = pickle.load(
                f, encoding='latin1')
        else:
            ground_template_filter_array_output = pickle.load(f)
    ground_template_filter_array(*ground_template_filter_array_input)
    assertIdenticalList(
        ground_template_filter_array_input,
        ground_template_filter_array_output,
        rtol)

    # debug
    return ground_template_filter_array_input, ground_template_filter_array_output

# Playground

In [ ]:
with open('tests/timestream/ground_template_filter_array_input.pkl', 'rb') as f:
    ground_template_filter_array_input = pickle.load(f)
with open('tests/timestream/ground_template_filter_array_output.pkl', 'rb') as f:
    ground_template_filter_array_output = pickle.load(f)

In [ ]:
for i in ground_template_filter_array_input:
    print(type(i), i)

In [ ]:
for i in ground_template_filter_array_output:
    print(type(i), i)

In [ ]:
for i, __ in enumerate(ground_template_filter_array_input):
    print(np.array_equal(ground_template_filter_array_input[i], ground_template_filter_array_output[i]))

In [ ]:
for i in ground_template_filter_array_input:
    if type(i) == np.ndarray:
        print(i.dtype)

In [ ]:
input_array, az, mask, pixel_size, groundmap, lr, filtmask = ground_template_filter_array_input

In [ ]:
input_array

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Input plot

In [ ]:
plt.plot(input_array[0])

In [ ]:
plt.plot(input_array[0] * mask[0])

In [ ]:
plt.plot(az)

In [ ]:
plt.plot(mask[0])

In [ ]:
plt.plot(filtmask[0])

In [ ]:
np.array_equal(mask[0], filtmask[0])

# Output

In [ ]:
%%timeit
output_array = input_array.copy()
ground_template_filter_array(output_array, az, mask, pixel_size)

In [ ]:
output_array = input_array.copy()
ground_template_filter_array(output_array, az, mask, pixel_size)

In [ ]:
plt.plot(output_array[0])

In [ ]:
test_ground_template_filter_array(rtol=1e-3)

# timeit

In [ ]:
def simulate_ground_input(nCh, nTime, nPix):
    az_time_width = 400
    az_min = 3.5
    az_max = 4.
    az_range = az_max - az_min

    input_array = (np.random.rand(nCh, nTime) - 0.5) * 0.2

    az = np.array([(az_range * (
        1 - abs(i % (2 * az_time_width) / float(az_time_width) - 1)
    ) + az_min) for i in range(nTime)])

    mask_half_width = 50
    mask = np.repeat([[(i % az_time_width < mask_half_width
                        or i % az_time_width > az_time_width - mask_half_width
                        ) for i in range(nTime)]], nCh, axis=0)

    pixel_size = az_range / nPix

    return input_array, az, mask, pixel_size

In [ ]:
temp=simulate_ground_input(100, 10000, 300)

In [ ]:
groundTime = %timeit -o ground_template_filter_array(*temp, lr=True)